In [2]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
import os
import numpy as np
import ast
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, accuracy_score, classification_report
import xgboost as xgb
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVR
from sklearn.neural_network import MLPRegressor


def get_files_by_keywords(directory, keywords):
    # 遍历指定文件夹，根据关键字数组提取文件。
    matched_files = []
    for filename in os.listdir(directory):
        if all(keyword in filename for keyword in keywords):
            matched_files.append(filename)
    return matched_files


def scaler_data(data_to_normalize, columns_to_normalize=["eirp", "seq_time"]):
    # 归一化数值列
    scaler = MinMaxScaler()
    data_scaled = data_to_normalize.copy()
    for column in columns_to_normalize:
        data_scaled[column] = data_scaled[column].astype(float)
    data_scaled.loc[:, columns_to_normalize] = scaler.fit_transform(data_scaled.loc[:, columns_to_normalize])
    return data_scaled


def process_array_string(array_string):
    # 去掉最大值、最小值并计算平均值
    # 如果输入是单个值，则直接返回
    if isinstance(array_string, (int, float)):
        return array_string
    # 将字符串转换为列表
    values = ast.literal_eval(array_string)
    # 如果列表长度大于2，去掉最大值和最小值
    if len(values) > 2:
        values.remove(max(values))
        values.remove(min(values))
    mean_value = np.mean(values)
    return mean_value


eirp_threshold = pd.read_excel("/Users/daylight/Desktop/macos/1Code/Competition/2024ShuMo/data/results/eirp值临界点统计.xlsx")

##### 修改这里 #####
project_root = "/Users/daylight/Desktop/macos/1Code/Competition/2024ShuMo"
data_dir = project_root + "/data/results/question1"
data_addcolumn_dir = project_root + "/data/results/question1_add_column"
os.makedirs(data_addcolumn_dir, exist_ok=True)
for index, row in eirp_threshold.iterrows():
    loc_id_value = str(row["loc_id"])
    nav_value = str(row["NAV"])
    protocol_value = str(row["protocol"])
    eirp_value = row["eirp分界点"]

    data_name = get_files_by_keywords(data_dir, ["training", loc_id_value, nav_value, "csv"])[0]
    data_path = data_dir + f"/{data_name}"
    data_df = pd.read_csv(data_path)
    # 添加 eirp_threshold 列，首先给出 False 默认值
    data_df["add_change"] = 0
    data_df.loc[(data_df["protocol"] == protocol_value) & (data_df["eirp"] >= eirp_value), "add_change"] = 1

    data_df.to_csv(data_addcolumn_dir + f"/{data_name}", index=False)